In [71]:
import numpy as np
k = "administraciones"

m = "infraestructuras"
print(len(m))

16


In [72]:
def int_miss(m):
    assert len(m) == 16, "El missatge ha de tenir 16 caràcters."
    m_int = np.array([ord(l) for l in m]).reshape(4,4)
    return m_int

k_int = int_miss(k)
m_int = int_miss(m)

print(k_int)
print(m_int)

[[ 97 100 109 105]
 [110 105 115 116]
 [114  97  99 105]
 [111 110 101 115]]
[[105 110 102 114]
 [ 97 101 115 116]
 [114 117  99 116]
 [117 114  97 115]]


In [73]:
# Funcions auxiliars

def tohex(a):
    return format(a, "02x")

In [74]:
# Constants

global sbox
global mixcolmat
global pol
global rcon

pol = int("100011011", 2)

sbox = np.array([
        0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
        0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
        0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
        0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
        0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
        0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
        0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
        0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
        0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
        0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
        0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
        0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
        0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
        0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
        0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
], dtype=np.uint8).reshape(16, 16)

mixcolmat =np.array([
    [0x02, 0x03, 0x01, 0x01],
    [0x01, 0x02, 0x03, 0x01],
    [0x01, 0x01, 0x02, 0x03],
    [0x03, 0x01, 0x01, 0x02]
], dtype=np.uint8)

rcon = np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36], dtype=np.uint8)

In [75]:
import numpy as np
m_int = np.array([[0x32, 0x88, 0x31, 0xe0],[0x43, 0x5a, 0x31, 0x37],[0xf6, 0x30, 0x98, 0x07],[0xa8, 0x8d, 0xa2, 0x34]])
k_int = np.array([[0x2b, 0x28, 0xab, 0x09],[0x7e, 0xae, 0xf7, 0xcf],[0x15, 0xd2, 0x15, 0x4f],[0x16, 0xa6, 0x88, 0x3c]])

def addroundkey(m, k):
    res = np.bitwise_xor(m, k)
    return res

mk = addroundkey(m_int, k_int)
print(tohex(mk))

[[ 50 136  49 224]
 [ 67  90  49  55]
 [246  48 152   7]
 [168 141 162  52]]
[[ 43  40 171   9]
 [126 174 247 207]
 [ 21 210  21  79]
 [ 22 166 136  60]]
[[ 25 160 154 233]
 [ 61 244 198 248]
 [227 226 141  72]
 [190  43  42   8]]


In [87]:
def map_sbox(el):
    d1 = int(el[0], 16)
    d2 = int(el[1], 16)
    return sbox[d1, d2]

def bytesub(mk):
    res = np.zeros((4, 4))
    mk_hex = np.vectorize(tohex)(mk)
    map = np.vectorize(map_sbox)(mk_hex)
    return map

bs = bytesub(mk)
print(np.vectorize(tohex)(bs))

[['d4' 'e0' 'b8' '1e']
 ['27' 'bf' 'b4' '41']
 ['11' '98' '5d' '52']
 ['ae' 'f1' 'e5' '30']]


In [94]:
def shiftrows(bs):
    res = np.zeros((4, 4))
    for i in range(4):
        res[i] = np.roll(bs[i], -i)
    return res.astype(int)

sr = shiftrows(bs)
print(np.vectorize(tohex)(sr))

[['d4' 'e0' 'b8' '1e']
 ['bf' 'b4' '41' '27']
 ['5d' '52' '11' '98']
 ['30' 'ae' 'f1' 'e5']]


In [78]:
def mult1(el):
    return el

print(mult1(7))
print(format(mult1(7), "08b"))

7
00000111


In [79]:
def mult2(el):
    bin = format(el, "08b")
    res = bin[1:] + "0"

    if bin[0] == "1":
        return (int(res, 2) ^ pol) % 256
    
    return int(res, 2)

print(mult2(7))
print(format(mult2(7), "08b"))

14
00001110


In [80]:
def mult3(el):
    m2 = mult2(el)
    return m2 ^ el

print(mult3(7))
print(format(mult3(7), "08b"))

9
00001001


In [81]:
def mixcolumns(sr):
    res = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            sum = 0
            for k in range(4):
                if mixcolmat[i][k] == 1:
                    sum ^= mult1(sr[k][j])
                elif mixcolmat[i][k] == 2:
                    sum ^= mult2(sr[k][j])
                elif mixcolmat[i][k] == 3:
                    sum ^= mult3(sr[k][j])
            res[i][j] = sum
    return res.astype(int)

In [83]:
def key_schedule(k, num_iteracions):
    num_iteraciones += 1
    if k.shape != (4, 4):
        raise ValueError("k ha de ser una matriu de 4x4!")
    w = np.zeros((num_iteracions, 4, 4), dtype=np.uint8)
    w[0, :, :] = k
    for n in range(1, num_iteracions):
        temp = w[n-1, :, 3].copy()  # copy the last column from the previous round key
        temp = np.roll(temp, -1)  # rotate the bytes
        temp = bytesub(temp)  # substitute bytes
        temp[0] ^= rcon[n-1]  # XOR the first byte with the rcon value
        w[n, :, 0] = np.bitwise_xor(temp, w[n-1, :, 0])  # XOR with the corresponding column from the previous round key
        for i in range(1, 4):
            w[n, :, i] = np.bitwise_xor(w[n, :, i-1], w[n-1, :, i])
    return w

In [104]:
print(np.vectorize(tohex)(key_schedule(k_int,10))[0, :, :])

[['2b' '28' 'ab' '09']
 ['7e' 'ae' 'f7' 'cf']
 ['15' 'd2' '15' '4f']
 ['16' 'a6' '88' '3c']]


In [115]:
def aes(m, k):
    ks = key_schedule(k, 10)
    for i in range(9):
        k = ks[i, :, :]
        #print(np.vectorize(tohex)(k))
        mk = addroundkey(m, k)
        #print(np.vectorize(tohex)(mk))
        bs = bytesub(mk)
        #print(np.vectorize(tohex)(bs))
        sr = shiftrows(bs)
        #print(np.vectorize(tohex)(sr))
        m = mixcolumns(sr)
        #print(np.vectorize(tohex)(m))

    k = ks[i + 1, :, :]
    mk = addroundkey(m, k)
    bs = bytesub(mk)
    sr = shiftrows(bs)

    k = ks[i + 2, :, :]
    res = addroundkey(sr, k)

    return res

print(np.vectorize(tohex)(aes(m_int, k_int)))

[['39' '02' 'dc' '19']
 ['25' 'dc' '11' '6a']
 ['84' '09' '85' '0b']
 ['1d' 'fb' '97' '32']]
